In [1]:
import xml.etree.ElementTree as ET
import re
import pandas as pd
import unicodedata
from docx import Document
from lxml import etree
import os
from win32com import client


In [2]:
from docx import Document

def read_docx(file_path):
    document = Document(file_path)
    text = []
    for paragraph in document.paragraphs:
        text.append(paragraph.text)
    return '\n'.join(text)

# Example usage
file_path = "MK.docx"
#file_path = "מס''ג לכל ספרי תרי עשר עם פרק ופסוק מעודכן העתק שמש 9.5.24.docx"

doc_text = read_docx(file_path)
print(doc_text)


הושע
פרק א
1 בארי – ב̇ את יהודית
יחזקיה – ב̇
2 תחלת – ג̇ וראש̇ פסו
בהושע – ב̇
וילדי – ל̇
זנה – ל̇
3 דבלים – ל̇
ותלד-לו בן – ב̇
4 ויאמר יהוה אליו – ו̇ ראש̇ פסו
ממלכות – ד̇ וכל יהושע
6 ותהר עוד ותלד – ז̇
7 והושעתים – ל̇
בקשת – ב̇
ובמלחמה – ג̇

פרק ב
1 לא-ימד – ל̇
ולא – י̇ז פסו̇ לא ולא לא מצע פסו
2 בני-יהודה – ל̇
ובני-ישראל – הי̇ וכל רא פס̇ כות
3 ולאחותיכם – ל̇
4 ותסר – ל̇
ונאפופיה – ל̇
5  אפשיטנה – ל̇
והצגתיה – ל̇
כיום – יא̇
הולדה – ל̇
כמדבר – ו̇
ושתה – ל̇
כארץ – ה̇
והמתיה – ל̇
6 ואת-בניה – ל̇
7 הורתם – ל̇
צמרי – ב̇
ושקויי – ל̇
8 שך – ל̇
בסירים – ל̇
גדרה – ל̇
9 ורדפה – ל̇
ולא-תשיג – ל̇
ובקשתם – ל̇
ולא תמצא – ל̇
10 אנכי נתתי – ג̇
והתירוש – ג̇
11 ופשתי – ב̇
12 אגלה – ב̇
נבלתה – ל̇ וחס
13 משושה – ל̇
חגה – ל̇
חדשה – ל̇
ושבתה – ל̇
מועדה – ל̇
14 אתנה – ל̇
ואכלתם – ב̇
15 ותעד – ל̇
נזמה – ל̇
וחליתה – ל̇
16 מפתיה – ל̇
והלכתיה – ל̇ וחס
17 ואת-עמק – ל̇
וענתה – ד̇
וכיום – ל̇
עלותה – ג̇
20 אשבור – ל̇ מל
והשכבתים – ל̇
21 וארשתיך – ג̇
וברחמים – ב̇
22 וידעת – ז̇
23 והיה – ו̇ בט̇ בסיפ̇
24 והארץ – ח̇ ר פ


In [3]:
lines = doc_text.split(sep="\n")

In [32]:
print(lines[-3])

In [4]:
import pandas as pd
import re

def parse_entries(lines):
    parsed_entries = []
    current_chapter = None
    current_verse = None
    current_book = None

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Check if the line indicates a new book, including "ספר"
        book_match = re.match(r'^ספר\s*[א-ת]+$', line) or re.match(r'^[א-ת]+$', line)
        if book_match and not re.match(r'^פרק\s+[א-ת]', line):
            current_book = line.strip()
            current_chapter = None
            current_verse = None
            continue

        # Check if the line indicates a new chapter
        chapter_match = re.match(r'^פרק\s+[א-ת]', line)
        if chapter_match:
            current_chapter = line.strip()
            current_verse = None
            continue

        # Extract verse number if present
        verse_match = re.match(r'^(\d+|\(\w+\))\s+', line)
        if verse_match:
            current_verse = verse_match.group(1)
            line = line[verse_match.end():].strip()

        # If no verse number is found, use the previous one
        if not current_verse:
            continue

#         # Extract the parts of the line separated by '–'
#         parts = re.split(r'\s+–?-?\s+', line)
#         parts = [p.strip() for p in parts]

        # Create a dictionary entry for the parsed line
        entry = {
            'book': current_book,
            'chapter': current_chapter,
            'verse': current_verse,
            'content': line
        }
        parsed_entries.append(entry)

    return parsed_entries

parsed_entries = parse_entries(lines)
df = pd.DataFrame(parsed_entries)

df


,book,chapter,verse,content
0,הושע,פרק א,1,בארי – ב̇ את יהודית
1,הושע,פרק א,1,יחזקיה – ב̇
2,הושע,פרק א,2,תחלת – ג̇ וראש̇ פסו
3,הושע,פרק א,2,בהושע – ב̇
4,הושע,פרק א,2,וילדי – ל̇
...,...,...,...,...
1648,ספר מלאכי,פרק ג,(כג),אליה - ה̇
1649,ספר מלאכי,פרק ג,(כג),בוא - ב̇ מל בסיפ̇
1650,ספר מלאכי,פרק ג,(כד),חרם - ב̇ סופ פסו בסיפ̇
1651,ספר מלאכי,פרק ג,(כד),נ̇ה̇ (סכום הפסוקים במלאכי)


In [5]:
import re
import pandas as pd

def break_down_entry_part_apparatus(part):
    # Initialize entry dictionary
    entry_dict = {'lemma': None, 'reading': None, 'comments': None}
    
    # Handle comments in parentheses
    comment_match = re.search(r'\((.*?)\)', part)
    if comment_match:
        entry_dict['comments'] = comment_match.group(1)
        part = re.sub(r'\(.*?\)', '', part).strip()
    
    # Split part into lemma and reading if '–' is present
    if '–' in part:
        split_part = re.split(r'\s*–\s*', part)
        if len(split_part) > 1:
            entry_dict['lemma'] = split_part[0].strip()
            entry_dict['reading'] = split_part[1].strip()
        else:
            entry_dict['reading'] = split_part[0].strip()
    else:
        # If there's no '–', handle hyphens within the lemma correctly
        split_part = re.split(r'\s*-\s*', part)
        if len(split_part) > 1:
            # Assume the first part before the first hyphen is the lemma, rest is the reading
            entry_dict['lemma'] = split_part[0].strip()
            entry_dict['reading'] = ' '.join(split_part[1:]).strip()
        else:
            entry_dict['reading'] = split_part[0].strip()
    
    return entry_dict

def process_decoded_content_apparatus(decoded_contents):
    processed_entries = []
    for decoded_content in decoded_contents:
        for part in decoded_content:
            entry = break_down_entry_part_apparatus(part)
            processed_entries.append(entry)
    return processed_entries


df['processed_content'] = df['content'].apply(break_down_entry_part_apparatus)

df

,book,chapter,verse,content,processed_content
0,הושע,פרק א,1,בארי – ב̇ את יהודית,"{'lemma': 'בארי', 'reading': 'ב̇ את יהודית', '..."
1,הושע,פרק א,1,יחזקיה – ב̇,"{'lemma': 'יחזקיה', 'reading': 'ב̇', 'comments..."
2,הושע,פרק א,2,תחלת – ג̇ וראש̇ פסו,"{'lemma': 'תחלת', 'reading': 'ג̇ וראש̇ פסו', '..."
3,הושע,פרק א,2,בהושע – ב̇,"{'lemma': 'בהושע', 'reading': 'ב̇', 'comments'..."
4,הושע,פרק א,2,וילדי – ל̇,"{'lemma': 'וילדי', 'reading': 'ל̇', 'comments'..."
...,...,...,...,...,...
1648,ספר מלאכי,פרק ג,(כג),אליה - ה̇,"{'lemma': 'אליה', 'reading': 'ה̇', 'comments':..."
1649,ספר מלאכי,פרק ג,(כג),בוא - ב̇ מל בסיפ̇,"{'lemma': 'בוא', 'reading': 'ב̇ מל בסיפ̇', 'co..."
1650,ספר מלאכי,פרק ג,(כד),חרם - ב̇ סופ פסו בסיפ̇,"{'lemma': 'חרם', 'reading': 'ב̇ סופ פסו בסיפ̇'..."
1651,ספר מלאכי,פרק ג,(כד),נ̇ה̇ (סכום הפסוקים במלאכי),"{'lemma': None, 'reading': 'נ̇ה̇', 'comments':..."


In [10]:
df["processed_content"][0:50].values

array([{'lemma': 'בארי', 'reading': 'ב̇ את יהודית', 'comments': None},
       {'lemma': 'יחזקיה', 'reading': 'ב̇', 'comments': None},
       {'lemma': 'תחלת', 'reading': 'ג̇ וראש̇ פסו', 'comments': None},
       {'lemma': 'בהושע', 'reading': 'ב̇', 'comments': None},
       {'lemma': 'וילדי', 'reading': 'ל̇', 'comments': None},
       {'lemma': 'זנה', 'reading': 'ל̇', 'comments': None},
       {'lemma': 'דבלים', 'reading': 'ל̇', 'comments': None},
       {'lemma': 'ותלד-לו בן', 'reading': 'ב̇', 'comments': None},
       {'lemma': 'ויאמר יהוה אליו', 'reading': 'ו̇ ראש̇ פסו', 'comments': None},
       {'lemma': 'ממלכות', 'reading': 'ד̇ וכל יהושע', 'comments': None},
       {'lemma': 'ותהר עוד ותלד', 'reading': 'ז̇', 'comments': None},
       {'lemma': 'והושעתים', 'reading': 'ל̇', 'comments': None},
       {'lemma': 'בקשת', 'reading': 'ב̇', 'comments': None},
       {'lemma': 'ובמלחמה', 'reading': 'ג̇', 'comments': None},
       {'lemma': 'לא-ימד', 'reading': 'ל̇', 'comments': None},
     

In [74]:
import re
import pandas as pd

def parse_entries(lines):
    parsed_entries = []
    current_chapter = None
    current_verse = None
    current_book = None

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Check if the line indicates a new chapter
        chapter_match = re.match(r'^פרק\s+[א-ת]', line)
        if chapter_match:
            current_chapter = line
            current_verse = None
            continue
        
        # Check if the line indicates a new book
        book_match = re.match(r'^[א-ת]+$', line)
        if book_match:
            current_book = line
            current_chapter = None
            current_verse = None
            continue

        # Extract verse number if present
        verse_match = re.match(r'^(\(\d+\)|\(\w+\)|\d+)\s+', line)
        if verse_match:
            current_verse = verse_match.group(1)
            line = line[verse_match.end():].strip()
        
        # If no verse number is found, use the previous one
        if not current_verse:
            continue

        # Extract the parts of the line
        parts = re.split(r'\s+◦\s+', line)
        parts = [p.strip() for p in parts]
        
        # Create a dictionary entry for the parsed line
        entry = {
            'book': current_book,
            'chapter': current_chapter,
            'verse': current_verse,
            'content': parts
        }
        parsed_entries.append(entry)

    return parsed_entries

def decode_content(contents):
    decoded_entries = []
    for content in contents:
        decoded_parts = re.split(r'\s{3,}|◦', content)
        decoded_parts = [part.strip() for part in decoded_parts if part.strip()]
        decoded_entries.append(decoded_parts)
    return decoded_entries

def break_down_entries(decoded_contents):
    broken_down_entries = []
    for decoded_content in decoded_contents:
        for entry in decoded_content:
            # Check for number of references with a dot on top
            ref_match = re.search(r'([א-ת]\̇)', entry)
            if ref_match:
                ref_position = ref_match.start()
                lemma = entry[:ref_position].strip() if ref_position > 0 else None
                references = entry[ref_position:].strip()
            else:
                lemma = None
                references = entry.strip()

            # Further break down references into parts
            ref_parts = re.split(r'\s+', references)
            num_references = ref_parts[0] if ref_parts else None
            comments = ' '.join(ref_parts[1:]) if len(ref_parts) > 1 else None

            broken_down_entries.append({
                'lemma': lemma,
                'num_references': num_references,
                'comments': comments
            })

    return broken_down_entries


parsed_entries = parse_entries(lines)
df = pd.DataFrame(parsed_entries)

# Apply the decoding function to the 'content' column
df['decoded_content'] = df['content'].apply(decode_content)

# Apply the break down function to the 'decoded_content' column
df['broken_down_content'] = df['decoded_content'].apply(break_down_entries)

df


,book,chapter,verse,content,decoded_content,broken_down_content
0,הושע,פרק א,1,[יחזקיה ב̇ אל הושע אל מיכה ◦],"[[יחזקיה, ב̇, אל הושע, אל מיכה]]","[{'lemma': None, 'num_references': 'יחזקיה', '..."
1,הושע,פרק א,2,[תחלת ג̇ ראש̇ פסוק̇ תחלת דבר יהוה תחלת ח...,"[[תחלת, ג̇ ראש̇ פסוק̇, תחלת דבר יהוה, תחלת חכמ...","[{'lemma': None, 'num_references': 'תחלת', 'co..."
2,הושע,פרק א,3,[ותלד לו בן ב̇ ויעבר האבל וילך ויקח ◦],"[[ותלד לו בן, ב̇, ויעבר האבל, וילך ויקח]]","[{'lemma': None, 'num_references': 'ותלד', 'co..."
3,הושע,פרק א,7,[ובמלחמה ג̇ או הנסה אלהים ואת בית יהודה ...,"[[ובמלחמה, ג̇, או הנסה אלהים, ואת בית יהודה, ב...","[{'lemma': None, 'num_references': 'ובמלחמה', ..."
4,הושע,פרק ב,9,[ח̇ פתח̇ מיחד̇ גנבתַם מצאתַם ובקשתַם א...,"[[ח̇ פתח̇ מיחד̇, גנבתַם, מצאתַם, ובקשתַם, אחזת...","[{'lemma': None, 'num_references': 'ח̇', 'comm..."
...,...,...,...,...,...,...
190,מלאכי,פרק ג,(יא),[תשכל ג̇ מחוץ וגערתי תפלט ◦],"[[תשכל, ג̇, מחוץ, וגערתי, תפלט]]","[{'lemma': None, 'num_references': 'תשכל', 'co..."
191,מלאכי,פרק ג,(כ),[ומרפא ג̇ הנני מעלה לה וזרחה לכם צוף ד...,"[[ומרפא, ג̇, הנני מעלה לה, וזרחה לכם, צוף דבש]]","[{'lemma': None, 'num_references': 'ומרפא', 'c..."
192,מלאכי,פרק ג,(כ),[ויצאתם ב̇ מן הכרמים ויצאתם ופשתם ◦],"[[ויצאתם, ב̇, מן הכרמים, ויצאתם ופשתם]]","[{'lemma': None, 'num_references': 'ויצאתם', '..."
193,מלאכי,פרק ג,(כב),[משה עבדי ג̇ מת רק חזק ואמץ זכרו תורת ◦],"[[משה עבדי, ג̇, מת, רק חזק ואמץ, זכרו תורת]]","[{'lemma': None, 'num_references': 'משה', 'com..."


In [25]:
# processing functions for sub-units of app_entry, for which there is matching lemma and verse data processed above

def lemma_verse_processor(text, chapter):
    # Regex to match the verse numbers at the beginning
    verse_regex = r'^(\d+(?:–\d+)?)\s'
    
    # Extract verses
    verses_match = re.match(verse_regex, text)
    if verses_match:
        verse_range = verses_match.group(1).split('–')
        if len(verse_range) == 2 and verse_range[0] != verse_range[1]:
            verses = {'from': int(verse_range[0]), 'to': int(verse_range[1])}
        else:
            verses = int(verse_range[0])
    else:
        verses = None
    
    # Isolate lemmas part by removing the verses
    lemmas_part = text[len(verses_match.group(0)):].strip() if verses_match else text
    
    return chapter, verses, process_lemma_with_range_and_diacritics(lemmas_part)

def parse_reading_entry(entry):
    # Refined regex pattern
    pattern = r"""
        \s?(?P<Sigla>[+<>~\.]*)                         # Captures special sigla
        \s*
        (?P<Reading>[^\(\)]*)                           # Captures reading (excluding parentheses)
        \s*
        (?P<Comment>\(.*\))?                            # Captures comments
    """

    # Compiling regex with VERBOSE flag for better readability and explanation
    compiled_pattern = re.compile(pattern, re.VERBOSE)
    match = compiled_pattern.match(entry)

    if not match:
        return None  # Return None if no match is found

    # Extracting groups into a dictionary
    parsed_entry = {k: v.strip() for k, v in match.groupdict().items() if v}

    # Special handling for comments containing specific markers
    if "Comment" in parsed_entry and any(marker in parsed_entry['Comment'] for marker in ["(ס)", "(פ)", "(>)"]):
        parsed_entry['Reading'] = parsed_entry.pop('Comment')

    return parsed_entry

def process_lemma_with_range_and_diacritics(lemma):
    # Adjust regex to include diacritical marks and punctuation within Hebrew words
    
    
    # Check for range indicated by "–" and process accordingly
    if "–" in lemma:
        from_lemma, to_lemma = lemma.split("–")
        return {
            'from': process_individual_lemma(from_lemma.strip()),
            'to': process_individual_lemma(to_lemma.strip())
        }
    if len(lemma.strip().split(' '))>1:#add lemma for a two word lemma separated by a space
        return{'lemma1':process_individual_lemma(lemma.strip().split(' ')[0]),
              'lemma2':process_individual_lemma(lemma.strip().split(' ')[1])}
    else:
        return process_individual_lemma(lemma.strip())


# Function to process individual lemmas or ranges, after the split,
lemma_regex = r'(\(?[^\d\(\)]+\)?)([\(\d\)]*)?(.+?׀)?'#(\d+(?:–\d+)?)\s

def process_individual_lemma(individual_lemma):
    matches = re.findall(lemma_regex, individual_lemma)
    processed_lemmas = []
#     for match in matches[:1]:
#         word, number = match
#         lemma_dict = {'lemma': word}
#         if number: lemma_dict['number'] = (number)
#         processed_lemmas.append(lemma_dict)

    for match in matches[:1]:
        word, number, paseq = match
        lemma_dict = {'lemma': word}
        if number: lemma_dict['number'] = (number)
        if paseq: lemma_dict['paseq'] = (paseq)
        processed_lemmas.append(lemma_dict)
    return processed_lemmas

def extract_cross_references(text): #extract cross-references
    # Regex to match some Roman numerals: sequences of "I"s followed by an optional "V"
    pattern = r'([I]*[V]?)'
    # Find all occurrences of the pattern
    found_numerals = re.findall(pattern, text)
    # Remove empty matches from the list
    found_numerals = [numeral for numeral in found_numerals if numeral]
    # Replace found Roman numerals with an empty string
    result_text = re.sub(pattern, '', text)
    return result_text, found_numerals

#split the string of the witnesses into each ms and if there is a pm\sm\tm as if there is a מ"ק\מ"ג also
def split_into_witnesses(text):
    witnesses = text.split()
    result = []
    mgk = None
    for witness in witnesses:
        # Splitting the witness based on the special character ־
        parts = witness.split('־')
        base_witness = parts[0]
        if len(parts)>1:
            mgk = parts[1]
        # Handling the ' character
        if base_witness.endswith("'") or base_witness.endswith("''") or base_witness.endswith("'''") or base_witness.endswith("!"):
            base_witness = base_witness.rstrip("'").rstrip("!")
            annotations = witness[len(base_witness):]
            result.append([base_witness, annotations])
        else:
            result.append([base_witness])
    if mgk:
        for ms in result:
            ms.append(mgk)
    return result

def process_input(input_str):
    if "=" in input_str and '(' in input_str:
        # Remove the '=' and return as "affirming variants"
        input_str = input_str.replace('(','').replace(')','')
        key, value = input_str.strip().split("=")
        return {"affirming variants": {key.strip(): value.strip()}}
    else:
        if "=" in input_str:
            # Return as "affirming and unidentified"
            key, value = input_str.split("=")
            return {"affirming and unidentified": {key.strip(): value.strip()}}
        else:
            # Return as comment"
            return {"comment": input_str.strip()}

def process_entry_variants(entry):
    variants = entry.split(';')
    split_variants = []
    for variant in variants:
        if ',' in variant:
            var, related_var = variant.split(',')#
            split_variants.append({'Type':'Variant','Info':var})
            split_variants.append({'Type':'Related Variant','Info':related_var})
        else:
            split_variants.append({'Type':'Variant','Info':variant})          
            
    processed_variants = []
    
    for variant_dict in split_variants:
        variant = variant_dict['Info']
#         print(type(variant['Info']))
        variant_type = variant_dict['Type']
        
        # Extract cross-references from the variant
        clean_variant, cross_reference = extract_cross_references(variant)

        if ':' in clean_variant:
            witnesses, reading = clean_variant.split(':', 1)
            #print(witnesses)
            if '=' in witnesses:
                witnesses = witnesses.replace('=', ' ') #if = in structured witnesses separated by :
                #print(witnesses)
            processed_variants.append({
                'Type': variant_type,
                'Witnesses': split_into_witnesses(witnesses),
                'Reading Info': parse_reading_entry(reading.strip()), #need to handle + and < as well as 
                'CrossReference': cross_reference
            })
        else: # this means that either there is a comment here in parenthesis, or a "= א" format in parenthesis.
            decoded_input = process_input(clean_variant)
            #if 'comment' in 
            if 'comment' in list(decoded_input.keys()):
                processed_variants.append({
                    'Comment': decoded_input['comment'],
                    'CrossReference': cross_reference
                })

            else:
                if "affirming variants" in list(decoded_input.keys()):
                    processed_variants.append({
                        'Type': 'Affirming Variant',
                        'Witnesses': split_into_witnesses(list(decoded_input['affirming variants'].keys())[0]),
                        'CrossReference': cross_reference
                    })
                else: #its a "affirming and unidentified" type, which means there are two variants here
                    witnesses = split_into_witnesses(list(decoded_input['affirming and unidentified'].keys())[0])
                    processed_variants.append({
                        'Type': 'Affirming Variant',
                        'Witnesses': witnesses,
                        'CrossReference': cross_reference
                    })
                    processed_variants.append({
                        'Type': 'Unidentified Variant',
                        'Witnesses': (witnesses[0][0],"'"),
                        'CrossReference': cross_reference
                    })
                    

    return processed_variants


In [60]:
def process_apparatus_entries(entries, chapter):
    processed_entries = []
    current_verse = 1
    for entry in entries:
        if ']' in entry:
            verse_lemma, entry_content = entry.split(']', 1)
            #if digits, so there are verse numbers, if not, take the previous verse number
            match = re.match(r'^(\d+)(.*)', verse_lemma)
            if match:
                chapter, verses, lemmas = lemma_verse_processor(verse_lemma, chapter)
                processed_entries.append({'Verse': verses, 'Lemma Info': lemmas, 'Entry': process_entry_variants(entry_content.strip())})
                if type(verses)=='list':
                    current_verse = verses[-1]
                else:
                    current_verse = verses
            else: # no verse number, so take previous verse and process lemma separately
                processed_entries.append({'Verse': current_verse, 'Lemma Info': process_lemma_with_range_and_diacritics(verse_lemma), 'Entry': process_entry_variants(entry_content.strip())})
    
        else:# no lemma, but still maybe verse number
            match = re.match(r'^(\d+)(.*)', entry)
            if match:
                verse, clean_entry = match.group(1), match.group(2)
                processed_entries.append({'Verse': verse, 'Entry': process_entry_variants(clean_entry.strip())})
                current_verse = verse
            else: #no verse, use previous
                processed_entries.append({'Verse': current_verse, 'Entry': process_entry_variants(entry.strip())})

#         match = re.match(r'^(\d+)\s+(.*)', entry)
#         if match:
#             verse = match.group(1)
#             entry = match.group(2)

        

    return processed_entries


processed_entries = process_apparatus_entries(df["Entries"][1], 'five')
# for entry in processed_entries:
#     print(entry)

In [14]:
df["Entries"][1][-6]#.strip()

'22 וארשתיך – יהוה] ר!: כל הפסוק אינו מנוקד'

In [63]:
processed_entries[12]#["Entry"]

{'Verse': 5,
 'Lemma Info': [{'lemma': 'ושתה֙'}],
 'Entry': [{'Type': 'Variant',
   'Witnesses': [['ל18'],
    ['20', "'"],
    ['29', "'"],
    ['נ', "'"],
    ['פ'],
    ['ר'],
    ['ק1', "'"]],
   'Reading Info': {'Reading': 'ושתיה'},
   'CrossReference': []},
  {'Type': 'Variant',
   'Witnesses': [['ל'],
    ['ל18'],
    ['20', "'"],
    ['29'],
    ['30'],
    ['37'],
    ['נ'],
    ['ק'],
    ['מ']],
   'Reading Info': {'Reading': 'ושת֙', 'Comment': '(י)'},
   'CrossReference': []},
  {'Type': 'Affirming Variant',
   'Witnesses': [['ל20', "''"], ['ק1', "''"], ['ש']],
   'CrossReference': []}]}

In [45]:
processed_entries#[0]#["Entry"]#[16]['Reading']

[{'Verse': 1,
  'Lemma Info': [{'lemma': '(פ)'}],
  'Entry': [{'Type': 'Variant',
    'Witnesses': [['ל29'], ['37'], ['פ'], ['מ']],
    'Reading Info': {'Reading': '(ס)'},
    'CrossReference': []},
   {'Type': 'Variant',
    'Witnesses': [['ר']],
    'Reading Info': {'Reading': '(>)'},
    'CrossReference': []},
   {'Type': 'Variant',
    'Witnesses': [['ר', 'מ"ק']],
    'Reading Info': {'Reading': '"פרשׄ"'},
    'CrossReference': []}]},
 {'Verse': 1,
  'Lemma Info': [{'lemma': 'יושבי'}],
  'Entry': [{'Type': 'Variant',
    'Witnesses': [['פ'], ['ק1', "'"], ['ר']],
    'Reading Info': {'Reading': 'ישבי'},
    'CrossReference': []}]},
 {'Verse': 1,
  'Lemma Info': [{'lemma': 'ואֵֽין־דַּ֥עת'}],
  'Entry': [{'Type': 'Variant',
    'Witnesses': [['ל20', "'"]],
    'Reading Info': {'Reading': 'ואֵ֥ין דַּעת־'},
    'CrossReference': []},
   {'Type': 'Variant',
    'Witnesses': [['ב"א']],
    'Reading Info': {'Reading': 'ואֵֽין־דַּ֥עת'},
    'CrossReference': []},
   {'Type': 'Related Variant',

In [494]:
processed_entries[0]["Entry"][1]['Reading']

'(>)'

In [486]:
processed_entries[14]["Entry"][-6]['Reading']

'"נפשוֹ גׄ מטעיין וסימנהׄ..."'

In [485]:
processed_entries[14]["Entry"][1]['Reading']

'"<...>שו קׄ" + "נפשו קׄ <...>פשם כתׄ וחד מן <...>לין דכתבין <...>וׄ תיבוׄ ק וׄ"'

In [ ]:
#now need to process the reading, including sigla [+< <...>] and comments
#also catch special marks in witnesses, like !

In [ ]:
################################# old functions

In [121]:
def process_full_entry(text, previous_verse=None):
    lemma, part_entry = split_full_entry(text)
    lemma_dict = lemma_verse_processor(lemma)

    # Use the previous verse if the current verse list is empty
    if not lemma_dict['verses'] and previous_verse is not None:
        lemma_dict['verses'] = previous_verse

    # Split part_entry by '|'
    if '|' in part_entry:
        entry_parts = part_entry.split('|')
    else:
        entry_parts = [part_entry]

    # Initialize a list to hold all processed parts
    processed_parts = []

    # Process each part separately
    for part in entry_parts:
        # Split part by ',' not inside parentheses
        sub_parts = split_on_comma_not_in_parentheses(part)

        # Process each sub-part using process_comma_entry
        processed_sub_parts = [process_comma_entry(sub_part) for sub_part in sub_parts]

        # Concatenate processed sub-parts for each part
        processed_parts.append(processed_sub_parts)

    # Combine processed parts. Assuming you want them as a nested list
    decoded_entries = processed_parts

    # Return the lemma_dict and decoded_entries, along with the verses used for this entry
    return lemma_dict, decoded_entries, lemma_dict['verses']

def split_on_comma_not_in_parentheses(part):
    """
    Splits the string on ',' not inside parentheses.
    """
    sub_parts = []
    current_part = []
    paren_depth = 0  # Track depth of parentheses

    for char in part:
        if char == '(':
            paren_depth += 1
        elif char == ')':
            paren_depth -= 1
        elif char == ',' and paren_depth == 0:
            # At a top-level comma, split here
            sub_parts.append(''.join(current_part))
            current_part = []
            continue

        current_part.append(char)

    # Add the last part if there's any
    if current_part:
        sub_parts.append(''.join(current_part))

    return sub_parts

def split_full_entry(text):
    sliced_entry = text.split(sep=']')
    lemma, entry = sliced_entry
#         print(f"lemma: {lemma}")
#         print(f"entry: {entry}")
    return lemma, entry    

def lemma_verse_processor(text):
    # Simplified approach: first split into digits and lemmas
    # Regex to match the verse numbers at the beginning
    verse_regex = r'^(\d+(?:–\d+)?)\s'
    
    # Extract verses
    verses_match = re.match(verse_regex, text)
    verses = list(map(int, verses_match.group(1).split('–'))) if verses_match else []
    
    # Isolate lemmas part by removing the verses
    lemmas_part = text[len(verses_match.group(0)):].strip() if verses_match else text
    return {
        'verses': verses,
        'lemmas': process_lemma_with_range_and_diacritics(lemmas_part)
    }

# Function to process individual lemmas or ranges, after the split,
lemma_regex = r'(k|q)?\s*([^\d\s]+)(\d?\,?\d?)'#(\d+(?:–\d+)?)\s

def process_lemma_with_range_and_diacritics(lemma):
    # Adjust regex to include diacritical marks and punctuation within Hebrew words
    
    
    # Check for range indicated by "–" and process accordingly
    if "–" in lemma:
        from_lemma, to_lemma = lemma.split("–")
        return {
            'from': process_individual_lemma(from_lemma.strip()),
            'to': process_individual_lemma(to_lemma.strip())
        }

    # Split lemma if there are separate lemmas with "/"
    split_lemmas = re.split(r'\s*/\s*', lemma) if '/' in lemma else [lemma]
    
    processed_lemmas = []
    for split_lemma in split_lemmas:
        processed = process_individual_lemma(split_lemma)
        processed_lemmas.extend(processed)
    
    return processed_lemmas

def process_individual_lemma(individual_lemma):
    matches = re.findall(lemma_regex, individual_lemma)
    processed_lemmas = []
    for match in matches:
        prefix, word, number = match
        lemma_dict = {'lemma': word}
        if prefix: lemma_dict[prefix] = True
        if number: lemma_dict['number'] = (number)
        processed_lemmas.append(lemma_dict)
    return processed_lemmas

# processing functions for sub-units of app_entry, for which there is matching lemma and verse data processed above

def extract_cross_references(text): #extract cross-references
    # Regex to match some Roman numerals: sequences of "I"s followed by an optional "V"
    pattern = r'([I]*[V]?)'
    # Find all occurrences of the pattern
    found_numerals = re.findall(pattern, text)
    # Remove empty matches from the list
    found_numerals = [numeral for numeral in found_numerals if numeral]
    # Replace found Roman numerals with an empty string
    result_text = re.sub(pattern, '', text)
    return result_text, found_numerals

def parse_witnesses(text):
    pattern = re.compile(r'\s?([^\d]*?)?(\d+)\s?\(?([^\)\d.]+)?\)?', re.DOTALL | re.UNICODE)
    parts = re.findall(pattern, text)
    # Filter out empty tuples
    return [part for part in parts if any(part)]

def parse_comma_witnesses(text):
    pattern = re.compile(r'\s?([^\d]*?)?(\d*)?\s?\(?([^\)\d]+)?\)?', re.DOTALL | re.UNICODE)
    parts = re.findall(pattern, text)
    # Filter out empty tuples
    return [part for part in parts if any(part)]

def parse_reading_entry(entry):
    # Refined regex pattern
    pattern = r"""
        \s?(?P<Sigla>[+<>~\.]*)                         # Captures special sigla
        \s*
        (?P<Reading>(?:[kq]*\s?)[\u0590-\u05FF\uFB1D-\uFB4F\s.]*    # Hebrew reading, including 'k', 'q'
                   (?:/\s(?:[kq]?\s?)?[\u0590-\u05FF\uFB1D-\uFB4F\s.]*)?)  # Allows for 'k'/'q' followed by Hebrew, separated by '/'
        \s*
        \s*
        (?P<Comment>\(.*\))?                     # Captures comments
    """
    
    # Compiling regex with VERBOSE flag for better readability and explanation
    compiled_pattern = re.compile(pattern, re.VERBOSE)
    match = compiled_pattern.match(entry)

    if not match:
        return None  # Return None if no match is found

    # Extracting groups into a dictionary
    parsed_entry = {k: v for k, v in match.groupdict().items() if v}

    return parsed_entry

#splitting entry into witnesses and reading (if only one group assign to witnesses)
def witness_reading_splitter(text):
    pattern = re.compile(r'(.*?)?([\+<>~.]*\s?[kq\u0590-\u05FF]+)(.*)?', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()  # Returns a tuple with the three parts
    else:
        pattern = re.compile(r'(.*?)([\+<>~])(.*)?', re.DOTALL)
        match = pattern.match(text)
        if match:
            return match.groups()  # Returns a tuple with the three parts
        else:
            return text
        return text  # No divider matching the pattern was found


def process_entry(entry):
    clean_entry, cross_references = extract_cross_references(entry)
    split_entry = witness_reading_splitter(clean_entry)
    if type(split_entry) is tuple:
        witnesses = {'Witnesses': parse_witnesses(split_entry[0])}
        if len(split_entry) == 2:
            reading = parse_reading_entry(split_entry[1])
        else:  # there are 3 groups:
            reading = parse_reading_entry(split_entry[1] + split_entry[2])
    else:
        witnesses = {'Witnesses': parse_witnesses(split_entry)}
        reading = ''
    # Include "Cross References" only if the list is not empty
    result = [witnesses, {"Reading": reading}]
    if cross_references:
        result.append({"Cross References": cross_references})
    return result

def process_comma_entry(entry):
    clean_entry, cross_references = extract_cross_references(entry)
    split_entry = witness_reading_splitter(clean_entry)
    if type(split_entry) is tuple:
        witnesses = {'Witnesses': parse_comma_witnesses(split_entry[0])}
        if len(split_entry) == 2:
            reading = parse_reading_entry(split_entry[1])
        else:  # there are 3 groups:
            reading = parse_reading_entry(split_entry[1] + split_entry[2])
    else:
        witnesses = {'Witnesses': parse_comma_witnesses(split_entry)}
        reading = ''
    # Include "Cross References" only if the list is not empty
    result = [witnesses, {"Reading": reading}]
    if cross_references:
        result.append({"Cross References": cross_references})
    return result


In [ ]:
# iterate over full entries. get verse number from previous if needed. also get reading from lemma. and split on | ,

In [ ]:
#process lemma:
#split into digits and lemmas. then process each separately

In [738]:
[process_full_entry(example) for example in full_entries][-1]#[0]['verses']


({'verses': [10], 'lemmas': [{'lemma': 'ואסרם'}]},
 [[({'Witnesses': [('', '96', 'pm')]},
    {'Reading': {'Reading': 'יאשרם '}},
    {'Cross References': []})],
  [({'Witnesses': [('', '150', '')]},
    {'Reading': {'Reading': 'על'}},
    {'Cross References': []})]])

In [731]:
# old function, didnt take the splitting into commas consideration
# def process_full_entry(text):
#     lemma, part_entry = split_full_entry(text)
#     lemma_dict = lemma_verse_processor(lemma)
# #     if len(lemma_dict['verses'])==0: #get verse from previous entry
# #         lemma_dict['verses'] = 
        
#     #entry_units = split_entry_units # splits on | and ,
#     #for entry in entry_units:
#     decoded_entry = process_entry(part_entry)
#     return lemma_dict, decoded_entry

In [639]:
# processing functions for sub-units of app_entry, for which there is matching lemma and verse data processed above

def remove_and_list_roman_numerals(text): #extract cross-references
    # Regex to match some Roman numerals: sequences of "I"s followed by an optional "V"
    pattern = r'([I]*[V]?)'
    # Find all occurrences of the pattern
    found_numerals = re.findall(pattern, text)
    # Remove empty matches from the list
    found_numerals = [numeral for numeral in found_numerals if numeral]
    # Replace found Roman numerals with an empty string
    result_text = re.sub(pattern, '', text)
    return result_text, found_numerals

def custom_split_string(text): #process witnesses
    pattern = re.compile(r'\s?([^\d]*?)?(\d+)\s?\(?([^\)\d]+)?\)?', re.DOTALL|re.UNICODE)    
    #pattern = r'([^,\d]*?)?(\d+)\s?(\(([^\)]+)?\)?)?([\skq]?)+'
    parts = re.findall(pattern, text)
    return parts

def parse_reading_entry(entry):
    # Refined regex pattern
    pattern = r"""
        \s?(?P<Sigla>[+<>~]?)                         # Captures special sigla
        \s*
        (?P<Reading>(?:[kq]?\s?)?[\u0590-\u05FF\uFB1D-\uFB4F\s.]*    # Hebrew reading, including 'k', 'q'
                   (?:/\s(?:[kq]?\s?)?[\u0590-\u05FF\uFB1D-\uFB4F\s.]*)?)  # Allows for 'k'/'q' followed by Hebrew, separated by '/'
        \s*
        (?P<Comment>\(.*\))?                     # Captures comments
    """
    
    # Compiling regex with VERBOSE flag for better readability and explanation
    compiled_pattern = re.compile(pattern, re.VERBOSE)
    match = compiled_pattern.match(entry)

    if not match:
        return None  # Return None if no match is found

    # Extracting groups into a dictionary
    parsed_entry = {k: v for k, v in match.groupdict().items() if v}

    return parsed_entry

#splitting entry into witnesses and reading (if only one group assign to witnesses)
def split_string(text):
    pattern = re.compile(r'(.*?)?([\+<>]?\s?[kq\u0590-\u05FF]+)(.*)?', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()  # Returns a tuple with the three parts
    else:
        pattern = re.compile(r'(.*?)([\+<>])(.*)?', re.DOTALL)
        match = pattern.match(text)
        if match:
            return match.groups()  # Returns a tuple with the three parts
        else:
            return text
        return text  # No divider matching the pattern was found


def process_entry(entry):
    clean_entry, cross_references = remove_and_list_roman_numerals(entry)
    split_entry = split_string(clean_entry)
    if type(split_entry) is tuple:
        witnesses = {'Witnesses': custom_split_string(split_entry[0])}
        if len(split_entry)==2:
            reading = parse_reading_entry(split_entry[1])
        else: #there are 3 groups:
            reading = parse_reading_entry(split_entry[1]+split_entry[2])
    else:
        witnesses = {'Witnesses': custom_split_string(split_entry)}
        reading = ''
    return witnesses, {"Reading":reading}, {"Cross References":cross_references}


# for entry in sample_texts:
#     print(f"entry: {entry}")
#     clean_entry, cross_references = remove_and_list_roman_numerals(entry)
#     split_entry = split_string(clean_entry)
#     if type(split_entry) is tuple:
#         witnesses = {'witnesses': custom_split_string(split_entry[0])}
#         reading = parse_reading_entry(split_entry[1])
#         print(f"witnesses: {witnesses}")
#         print(f"reading: {reading}")
#     else:
#         witnesses = {'witnesses': custom_split_string(split_entry)}
#         print(f"witnesses: {witnesses}")
#     print(f"references: {cross_references}")


In [640]:
sample_texts = [
    "96 (non voc)",
    "30 (pm) 93 (pm) 150 (pm) + סךII IV (See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))",
    "93 (non voc) 96 150 (non voc) + את",
    "30 (pm) >",
    "30 + לי (non voc)I II",
    "30 (pm) >I II IV (similarly b. Pesaḥim 87bmss)",
    "93 (pm) ביהושעIV (similarly PesiqtaR 33 (153b))",
    "96 >I II IV",
    "130 k",
    "G-B Msr 34 k ממני / q ממנוIV",
    "93 כד..",
    "150 ..דברים",
    "G-B Eb 94 ותָעָד (understood as \עוד (rather than \עדי))",
    "30 89 (sm) 93 (pm) 150 (non voc) + כיI II IV"
]

In [643]:
process_entry(sample_texts[6])

({'Witnesses': [('', '93', 'pm')]},
 {'Reading': {'Reading': 'ביהושע ',
   'Comment': '(similarly PesiqtaR 33 (153b))'}},
 {'Cross References': ['IV']})

In [638]:
clean_entry, cross_references = remove_and_list_roman_numerals(sample_texts[6])
split_entry = split_string(clean_entry)
if type(split_entry) is tuple:
    witnesses = {'Witnesses': custom_split_string(split_entry[0])}
    if len(split_entry)==2:
        reading = parse_reading_entry(split_entry[1])
    else: #there are 3 groups:
        reading = parse_reading_entry(split_entry[1]+split_entry[2])

reading

{'Reading': 'ביהושע ', 'Comment': '(similarly PesiqtaR 33 (153b))'}

In [462]:
def custom_split_string(text): #process witnesses
    pattern = re.compile(r'\s?([^\d]*?)?(\d+)\s?\(?([^\)\d]+)?\)?', re.DOTALL|re.UNICODE)    
    #pattern = r'([^,\d]*?)?(\d+)\s?(\(([^\)]+)?\)?)?([\skq]?)+'
    parts = re.findall(pattern, text)
    return parts
# "96 (non voc)",
# "30 (pm) 93 (pm) 150 (pm)
test_witness = "30 (pm) 93 150 (pm)"
custom_split_string(test_witness)


[('', '30', 'pm'), ('', '93', ''), ('', '150', 'pm')]

In [623]:
#try parsing single entry app, splitting into witnesses and reading (if only one group assign to witnesses)
def split_string(text):
    pattern = re.compile(r'(.*?)?([\+<>]?\s?[kq\u0590-\u05FF]+)(.*)?', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()  # Returns a tuple with the three parts
    else:
        pattern = re.compile(r'(.*?)([\+<>])(.*)?', re.DOTALL)
        match = pattern.match(text)
        if match:
            return match.groups()  # Returns a tuple with the three parts
        else:
            return text
        return text  # No divider matching the pattern was found

# Example usage
processed_sample = [split_string(text) for text in sample_texts]

print(processed_sample)


['96 (non voc)', ('30 (pm) 93 (pm) 150 (pm) ', '+ סך', 'II IV (See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))'), ('93 (non voc) 96 150 (non voc) ', '+ את', ''), ('30 (pm) ', '>', ''), ('30 ', '+ לי', ' (non voc)I II'), ('30 (pm) ', '>', 'I II IV (similarly b. Pesaḥim 87bmss)'), ('93 (pm)', ' ביהושע', 'IV (similarly PesiqtaR 33 (153b))'), ('96 ', '>', 'I II IV'), ('130', ' k', ''), ('G-B Msr 34', ' k', ' ממני / q ממנוIV'), ('93', ' כד', '..'), ('150 ..', 'דברים', ''), ('G-B Eb 94', ' ותָעָד', ' (understood as \\עוד (rather than \\עדי))'), ('30 89 (sm) 93 (pm) 150 (non voc) ', '+ כי', 'I II IV')]


In [619]:
#function for parsing the reading+comment (assumes witnesses and cross references have been removed)
def parse_reading_entry(entry):
    # Refined regex pattern
    pattern = r"""
        \s?(?P<Sigla>[+<>~]?)                         # Captures special sigla
        \s*
        (?P<Reading>(?:[kq]?\s?)?[\u0590-\u05FF\uFB1D-\uFB4F\s.]*    # Hebrew reading, including 'k', 'q'
                   (?:/\s(?:[kq]?\s?)?[\u0590-\u05FF\uFB1D-\uFB4F\s.]*)?)  # Allows for 'k'/'q' followed by Hebrew, separated by '/'
        \s*
        (?P<Comment>\(.*\))?                     # Captures comments
    """
    
    # Compiling regex with VERBOSE flag for better readability and explanation
    compiled_pattern = re.compile(pattern, re.VERBOSE)
    match = compiled_pattern.match(entry)

    if not match:
        return None  # Return None if no match is found

    # Extracting groups into a dictionary
    parsed_entry = {k: v for k, v in match.groupdict().items() if v}

    return parsed_entry

# Process the sample reading texts with the refined function
parse_reading_entry = [parse_reading_entry(text) for text in sample_reading_texts]

parse_reading_entry


[{'Reading': 'סך ',
  'Comment': '(See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))'},
 {'Sigla': '+', 'Reading': 'את'},
 {'Sigla': '>'},
 {'Sigla': '+', 'Reading': 'לי ', 'Comment': '(non voc)'},
 {'Sigla': '>', 'Comment': '(similarly b. Pesaḥim 87bmss)'},
 {'Reading': 'ביהושע ', 'Comment': '(similarly PesiqtaR 33 (153b))'},
 {},
 {'Reading': 'k'},
 {'Reading': 'k ממני / q ממנו'},
 {'Reading': 'כד..'},
 {'Reading': '..דברים'},
 {'Reading': 'נַחֵם ',
  'Comment': '(taken as infinitive, see Yeivin, Babylonian Vocalization, 1:542)'},
 {'Reading': 'חכֵם ', 'Comment': '(!)'}]

In [587]:
import re

def custom_string_processor(input_string, regex_pattern):
    # Helper function to apply regex and extract groups
    def apply_regex_and_extract(text):
        matches = re.finditer(regex_pattern, text)
        results = []
        for match in matches:
            results.append({
                'witnesses': match.group(1).strip(),
                'reading': match.group(2).strip(),
                'comments': match.group(3).strip() if match.group(3) else ''
            })
        return results

    # Process splits with "|", then ","
    def process_splits(text, delimiter):
        parts = text.split(delimiter)
        processed_parts = []
        for part in parts:
            # Apply regex to each part
            processed = apply_regex_and_extract(part)
            if processed:
                processed_parts.extend(processed)
        return processed_parts

    # Start processing
    processed_result = process_splits(input_string, '|')  # Start with the highest level of split

    return processed_result

# Custom regex pattern as provided
custom_regex = r'^(.*?)([\+<~>]?\s?[\u0590-\u05FF]+.*?)(.*)$'

# Test with the provided sample input
sample_input = "G-B msr. 30 (pm) G-A 89 (sm?) 150 (non voc) k, 30 (sm) 89 (sm) 93 (sm) 96 150 (pm) q, 93 (pm) + שערורהIV II (bla bla (f)) | 150 >"
processed_sample = [custom_string_processor(text, custom_regex) for text in sample_texts]


print(processed_sample)

def split_string(text):
    pattern = re.compile(r'^(.*?)([\+<>]?\s?[\u0590-\u05FF]+.*?)(.*)$', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()  # Returns a tuple with the three parts
    else:
        return None  # No divider matching the pattern was found

# Example usage
# text = "G-B msr. 30 (pm) G-A 89 (pm?) 150 (sm) k, 30 (sm) 89 (sm) 93 (sm) 96 150 (pm) q, 93 (pm) > שערורהIV (bla bla (f))"#"30 89 (sm) 93 (pm) 150 (non voc) + כיI II IV"#"93 (pm) < ביהושעIV (similarly PesiqtaR 33 (153b))"
# split_parts = split_string(text)
# if split_parts:
#     print("witnesses:", split_parts[0])
#     print("reading:", split_parts[1])
#     print("After dividers:", split_parts[2])
# else:
#     print("No dividers found.")



[[], [{'witnesses': '30 (pm) 93 (pm) 150 (pm)', 'reading': '+ סך', 'comments': 'II IV (See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))'}], [{'witnesses': '93 (non voc) 96 150 (non voc)', 'reading': '+ את', 'comments': ''}], [], [{'witnesses': '30', 'reading': '+ לי', 'comments': '(non voc)I II'}], [], [{'witnesses': '93 (pm)', 'reading': 'ביהושע', 'comments': 'IV (similarly PesiqtaR 33 (153b))'}], [], [], [{'witnesses': 'G-B Msr 34 k', 'reading': 'ממני', 'comments': '/ q ממנוIV'}], [{'witnesses': '93', 'reading': 'כד', 'comments': '..'}], [{'witnesses': '150 ..', 'reading': 'דברים', 'comments': ''}], [{'witnesses': 'G-B Eb 94', 'reading': 'ותָעָד', 'comments': '(understood as \\עוד (rather than \\עדי))'}], [{'witnesses': '30 89 (sm) 93 (pm) 150 (non voc)', 'reading': '+ כי', 'comments': 'I II IV'}]]


In [37]:

def parse_apparatus_entry(entry):
    """Parse an apparatus entry into lemma(s) and content."""
    parts = entry.split(']')
    lemmas_contents = []
    for part in parts:
        if part.strip():
            lemma, content = part.split('[', 1) if '[' in part else (part, '')
            lemmas_contents.append((lemma.strip(), content.strip()))
    return lemmas_contents

def create_tei_document(apparatus_lines):
    """Create a TEI document from apparatus lines."""
    TEI_NAMESPACE = "http://www.tei-c.org/ns/1.0"
    TEI = "{%s}" % TEI_NAMESPACE
    NSMAP = {"tei": TEI_NAMESPACE}
    
    tei_root = ET.Element(TEI+"TEI", nsmap=NSMAP)
    tei_header = ET.SubElement(tei_root, TEI+"teiHeader")
    text = ET.SubElement(tei_root, TEI+"text")
    body = ET.SubElement(text, TEI+"body")
    current_chapter = None
    last_verse = None
    
    for line in apparatus_lines:
        line = line.strip()
        if not line:
            continue
        if line.startswith('Chapter'):
            chapter_number = line.split(' ')[1]
            current_chapter = ET.SubElement(body, TEI+"div", type="chapter", n=chapter_number)
            last_verse = None
            continue
        # Use regex to check if the line starts with a verse number and capture it
        match = re.match(r"^(\d+)\s*(.*)", line)
        if match:
            verse_number, entry = match.groups()
            last_verse = verse_number
        else:
            entry = line
            verse_number = last_verse
        
        if current_chapter is not None and verse_number:
            lemmas_contents = parse_apparatus_entry(entry)
            for lemma, content in lemmas_contents:
                app = ET.SubElement(current_chapter, TEI+"app")
                lem = ET.SubElement(app, TEI+"lem", n=verse_number)
                lem.text = lemma
                if content:
                    rdg = ET.SubElement(app, TEI+"rdg")
                    rdg.text = content

    return ET.ElementTree(tei_root)

def save_tei_file(tree, filename):
    """Save the TEI XML tree to a file."""
    tree.write(filename, encoding="UTF-8", xml_declaration=True, method="xml", short_empty_elements=True)



# Replace 'your_input_file.txt' with the path to your actual input file
input_file = '01 Hosea App III - מתוקן.txt'
output_file = 'apparatus_tei.xml'

with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

tei_tree = create_tei_document(lines)
save_tei_file(tei_tree, output_file)

print(f"TEI document has been saved to {output_file}.")


TEI document has been saved to apparatus_tei.xml.


In [47]:
import xml.etree.ElementTree as ET
import re

def create_tei_document(apparatus_lines):
    TEI_NAMESPACE = "http://www.tei-c.org/ns/1.0"
    TEI = "{%s}" % TEI_NAMESPACE
    
    tei_root = ET.Element(TEI + "TEI", xmlns=TEI_NAMESPACE)
    tei_header = ET.SubElement(tei_root, TEI + "teiHeader")
    text = ET.SubElement(tei_root, TEI + "text")
    body = ET.SubElement(text, TEI + "body")
    current_chapter = None
    last_verse_number = None
    
    for line in apparatus_lines:
        line = line.strip()
        if not line:
            continue
        if line.startswith('Chapter'):
            chapter_number = line.split(' ')[1].strip()
            current_chapter = ET.SubElement(body, TEI + "div", type="chapter", n=chapter_number)
        else:
            # Attempt to extract verse number and lemma content
            parts = re.match(r"^(\d+)\s*(.*)", line)
            if parts:
                verse_number, remainder = parts.groups()
                last_verse_number = verse_number  # Update last verse number with current
                
                # Further split to separate lemma from variants, if present
                lemma_section, variants_section = remainder.split(']', 1) if ']' in remainder else (remainder, "")
                lemma_section = lemma_section.strip()
                variants_section = variants_section.strip()

                if current_chapter is not None and verse_number:
                    # Create an apparatus entry for the lemma
                    app = ET.SubElement(current_chapter, TEI + "app")
                    lem = ET.SubElement(app, TEI + "lem", n=verse_number)
                    lem.text = lemma_section
                    
                    # Add variant readings if present
                    if variants_section:
                        rdg = ET.SubElement(app, TEI + "rdg")
                        rdg.text = variants_section
            else:
                print(f"Line does not conform to expected format: {line}")

    return ET.ElementTree(tei_root)

def save_tei_file(tree, filename):
    tree.write(filename, encoding="UTF-8", xml_declaration=True, method="xml", short_empty_elements=True)

# Replace 'your_input_file.txt' with the path to your actual input file
input_file = '01 Hosea App III - מתוקן.txt'
output_file = 'apparatus_tei.xml'

with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

tei_tree = create_tei_document(lines)
save_tei_file(tei_tree, output_file)

print(f"TEI document has been saved to {output_file}.")


Line does not conform to expected format: ﻿App III: Hosea
Line does not conform to expected format: יחזקיה] 30 93 (pm) 96 יחזקיהו
Line does not conform to expected format: ירבעם בן] 30 + נבט (non voc)
Line does not conform to expected format: לו] 96 >I  II IV
Line does not conform to expected format: ממלכוּת] 96 ממלכוֹת
Line does not conform to expected format: יזרעאל] 150 ישראל (parall; but 150-Tg: יזרעאל)
Line does not conform to expected format: כי] 93 (pm) + את
Line does not conform to expected format: אוסיף] 150 (pm) >
Line does not conform to expected format: את] 93 (pm) >
Line does not conform to expected format: בסוסים] 96 ובסוסיםI IV
Line does not conform to expected format: אשר2] 96 + לא
Line does not conform to expected format: אחד] 30 (pm) 150 (pm) >
Line does not conform to expected format: והצגתיה] 93 (pm) + כיום ערומה והצגתיה
Line does not conform to expected format: ושתִּה] 150 (pm) ושמתיה
Line does not conform to expected format: כי] 150 (non voc) + כי
Line does not co

In [10]:
import xml.etree.ElementTree as ET
import re


def create_apparatus_entry(verse_number, content, TEI):
    """Create TEI element for an apparatus entry."""
    app = ET.Element(TEI + "app")
    
    # Extract lemma text and the rest (witnesses, variant reading, and comments)
    lemma_text, _, rest = content.partition(']')
    lem = ET.SubElement(app, TEI + "lem")
    lem.text = lemma_text.strip()
    
    # Extract comments
    comments = re.findall(r'\((.*?)\)', rest)
    for comment in comments:
        note = ET.SubElement(app, TEI + "note")
        note.text = comment
    
    # Remove comments from rest for further processing
    rest = re.sub(r'\(.*?\)', '', rest).strip()
    
    # Extract and process witnesses and cross-references
    if rest:
        rdg = ET.SubElement(app, TEI + "rdg")
        witnesses, _, variant_reading = rest.partition(' ')
        if witnesses:
            rdg.set('wit', witnesses.strip())
        if variant_reading:
            rdg.text = variant_reading.strip()
        
        # Extract cross-references, assuming they are indicated by Roman numerals at the start
        cross_refs = re.findall(r'\bI{1,3}V?|\bIV', rest)
        for ref in cross_refs:
            ref_element = ET.SubElement(rdg, TEI + "ref")
            ref_element.set('target', '#' + ref)  # Assuming target IDs are prefixed with '#'
            ref_element.text = "See apparatus entry " + ref
    
    return app

def create_tei_document(apparatus_lines):
    TEI_NAMESPACE = "http://www.tei-c.org/ns/1.0"
    TEI = "{%s}" % TEI_NAMESPACE
    root = ET.Element(TEI + "TEI", xmlns=TEI_NAMESPACE)
    header = ET.SubElement(root, TEI + "teiHeader")
    text = ET.SubElement(root, TEI + "text")
    body = ET.SubElement(text, TEI + "body")
    div = ET.SubElement(body, TEI + "div")
    
    last_verse_number = None
    for line in apparatus_lines:
        line = line.strip()
        if not line:
            continue
        
        # Determine if the line starts with a verse number
        match = re.match(r'^(\d+)', line)
        if match:
            last_verse_number = match.group(1)
            content = line[len(last_verse_number):].strip()
        else:
            content = line
        
        if last_verse_number:
            entry = create_apparatus_entry(last_verse_number, content, TEI)
            div.append(entry)
    
    return ET.ElementTree(root)

def save_tei_file(tree, filename):
    tree.write(filename, encoding="UTF-8", xml_declaration=True, method="xml")


# Replace 'your_input_file.txt' with the path to your actual input file
input_file = '01 Hosea App III - מתוקן.txt'
output_file = 'apparatus_tei.xml'

with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

tei_tree = create_tei_document(lines)
save_tei_file(tei_tree, output_file)

print(f"TEI document has been saved to {output_file}.")


TEI document has been saved to apparatus_tei.xml.


In [232]:
#split entry into witnesses, reading, and comments 

def split_string(text):
    pattern = re.compile(r'^(.*?)([\+<>]?\s?[\u0590-\u05FF]+.*?)(.*)$', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()  # Returns a tuple with the three parts
    else:
        return None  # No divider matching the pattern was found

# Example usage
text = "G-B msr. 30 (pm) G-A 89 (pm?) 150 (sm) k, 30 (sm) 89 (sm) 93 (sm) 96 150 (pm) q, 93 (pm) > שערורהIV (bla bla (f))"#"30 89 (sm) 93 (pm) 150 (non voc) + כיI II IV"#"93 (pm) < ביהושעIV (similarly PesiqtaR 33 (153b))"
split_parts = split_string(text)
if split_parts:
    print("witnesses:", split_parts[0])
    print("reading:", split_parts[1])
    print("After dividers:", split_parts[2])
else:
    print("No dividers found.")


witnesses: G-B msr. 30 (pm) G-A 89 (pm?) 150 (sm) k, 30 (sm) 89 (sm) 93 (sm) 96 150 (pm) q, 93 (pm) 
reading: > שערורה
After dividers: IV (bla bla (f))


[('G-B msr. ', '30', 'pm'), ('G-A ', '89', 'pm?'), ('', '150', 'non voc'), ('', '30', 'sm'), ('', '89', 'sm'), ('', '93', 'sm'), ('MS-G ', '150', 'pm')]


In [161]:
#parse comments
import re

def remove_and_list_roman_numerals(text):
    # Regex to match some Roman numerals: sequences of "I"s followed by an optional "V"
    pattern = r'([I]*[V]?)'
    # Find all occurrences of the pattern
    found_numerals = re.findall(pattern, text)
    # Remove empty matches from the list
    found_numerals = [numeral for numeral in found_numerals if numeral]
    # Replace found Roman numerals with an empty string
    result_text = re.sub(pattern, '', text)
    return result_text, found_numerals

# Example usage
text = "II IV (See b. R.HaŠanamss 23b, LamR Buber 1:16 (40b))"
result_text, numerals_found = remove_and_list_roman_numerals(text)
print("Modified text:", result_text.strip())
print("Numerals found:", numerals_found)



Modified text: (See b. R.HaŠanamss 23b, LamR Buber 1:16 (40b))
Numerals found: ['II', 'IV']


In [178]:
def process_entry(entry):
    split_parts = split_string(entry)
    if split_parts is None:
        return None
    
    witnesses, reading, comments = split_parts

    structured_entry = {
        'witnesses': [],
        'reading': reading,
        'comments': '',
        'cross_references': []
    }

    for part in custom_split_string(witnesses):
        # Assuming part[1] contains the witness number and part[0], part[2], part[3] contain additional info
        witness_info = {
            'n': part[1],
            'text': f"{part[0]}{part[1]} {part[2].strip()}{part[3]}"
        }
        structured_entry['witnesses'].append(witness_info)

    comments_text, numerals_found = remove_and_list_roman_numerals(comments)
    structured_entry['comments'] = comments_text
    structured_entry['cross_references'] = numerals_found

    return structured_entry

def create_apparatus_entry(verse_number, content, TEI):
    """Create TEI element for an apparatus entry."""
    TEI_ns = {'tei': TEI}  # Define the namespace dictionary if needed
    app = ET.Element(f"{{{TEI}}}app")  # Using namespace in the tag
    
    # Extract lemma text and the rest (witnesses, variant reading, and comments)
    lemma_text, _, rest = content.partition(']')
    lem = ET.SubElement(app, f"{{{TEI}}}lem")
    lem.text = lemma_text.strip('[] ')

    structured_entry = process_entry(rest)
    if not structured_entry:
        return None

    for witness in structured_entry['witnesses']:
        wit_element = ET.SubElement(app, f"{{{TEI}}}wit", {'n': witness['n']})
        wit_element.text = witness['text']
    
    rdg_element = ET.SubElement(app, f"{{{TEI}}}rdg")
    rdg_element.text = structured_entry['reading']

    if structured_entry['comments']:
        comment_element = ET.SubElement(app, f"{{{TEI}}}note")
        comment_element.text = structured_entry['comments']

    for ref in structured_entry['cross_references']:
        ref_element = ET.SubElement(app, f"{{{TEI}}}ref")
        ref_element.text = ref

    return app


In [181]:
import xml.etree.ElementTree as ET
import re


def create_tei_document(apparatus_lines):
    TEI_NAMESPACE = "http://www.tei-c.org/ns/1.0"
    ET.register_namespace('', TEI_NAMESPACE)  # Register the default namespace

    # Create the root element without redundantly specifying the xmlns attribute
    root = ET.Element("{%s}TEI" % TEI_NAMESPACE)
    header = ET.SubElement(root, "{%s}teiHeader" % TEI_NAMESPACE)
    text = ET.SubElement(root, "{%s}text" % TEI_NAMESPACE)
    body = ET.SubElement(text, "{%s}body" % TEI_NAMESPACE)
    div = ET.SubElement(body, "{%s}div" % TEI_NAMESPACE)
    
    last_verse_number = None
    for line in apparatus_lines:
        line = line.strip()
        if not line:
            continue
        
        # Determine if the line starts with a verse number
        match = re.match(r'^(\d+)', line)
        if match:
            last_verse_number = match.group(1)
            content = line[len(last_verse_number):].strip()
        else:
            content = line
        
        if last_verse_number:
            entry = create_apparatus_entry(last_verse_number, content, TEI_NAMESPACE)
            if entry is not None:  # Ensure entry creation was successful
                div.append(entry)
    
    return ET.ElementTree(root)

def save_tei_file(tree, filename):
    tree.write(filename, encoding="UTF-8", xml_declaration=True, method="xml")


# Replace 'your_input_file.txt' with the path to your actual input file
input_file = '01 Hosea App III - מתוקן.txt'
output_file = 'apparatus_tei.xml'

with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

tei_tree = create_tei_document(lines)
save_tei_file(tei_tree, output_file)

print(f"TEI document has been saved to {output_file}.")


TEI document has been saved to apparatus_tei.xml.


In [227]:
import re

def split_string(text):
    pattern = re.compile(r'^(.*?)([\+<~>]?\s?[\u0590-\u05FF]+.*?)(.*)$', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()
    else:
        return None

def custom_split_string(text):
    pattern = re.compile(r'([^,\d]*?)?(\d+)\s?(\([^\)]+\)?)?([\skq]*)?', re.DOTALL|re.UNICODE)
    parts = re.findall(pattern, text)
    return parts

def remove_and_list_roman_numerals(text):
    pattern = r'([I]*[V]?)'
    found_numerals = re.findall(pattern, text)
    found_numerals = [numeral for numeral in found_numerals if numeral]
    result_text = re.sub(pattern, '', text)
    return result_text, found_numerals

# def process_entry(entry):
#     split_parts = split_string(entry)
#     if split_parts is None:
#         return "Unable to process entry: No valid dividers found."
    
#     witnesses, reading, comments = split_parts

#     witness_entries = []
#     for part in custom_split_string(witnesses):
#         witness_entry = f'<witness n="{part[1]}">{part[0]}{part[1]} {part[2].strip()}{part[3]}</witness>'
#         witness_entries.append(witness_entry)
#     witnesses_tagged = "\n".join(witness_entries)

#     reading_tagged = f'<reading>{reading}</reading>'

#     comments_text, numerals_found = remove_and_list_roman_numerals(comments)
#     comments_tagged = f'<comment>{comments_text}</comment>'
#     cross_references = "\n".join([f'<ref>{numeral}</ref>' for numeral in numerals_found])

#     # Combine all parts, placing cross_references outside the comment
#     tei_entry = f"{witnesses_tagged}\n{reading_tagged}\n{comments_tagged}\n{cross_references}"
#     return tei_entry

# Example usage
entry ="G-B msr. 30 (pm) G-A 89 (pm?) 150 (non voc) k, 30 (sm) 89 (sm) 93 (sm) 96 150 (pm) q, 93 (pm) > שערורהIV II (bla bla (f))"
processed_entry = process_entry(entry)
print(processed_entry)


{'witnesses': [{'n': '30', 'text': 'G-B msr. 30 (pm) '}, {'n': '89', 'text': 'G-A 89 (pm?) '}, {'n': '150', 'text': '150 (non voc) k'}, {'n': '30', 'text': ' 30 (sm) '}, {'n': '89', 'text': '89 (sm) '}, {'n': '93', 'text': '93 (sm) '}, {'n': '96', 'text': '96 '}, {'n': '150', 'text': '150 (pm) q'}, {'n': '93', 'text': ' 93 (pm) '}], 'reading': '> שערורה', 'comments': '  (bla bla (f))', 'cross_references': ['IV', 'II']}


In [60]:
rest = "93 (pm) + ביהושעIV (similarly PesiqtaR 33 (153b))"
rest

'93 (pm) + ביהושעIV (similarly PesiqtaR 33 (153b))'

In [ ]:
#### 

In [ ]:
####### old stuff

In [41]:

def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def strip_non_hebrew(word):
    normalized_word = unicodedata.normalize('NFD', word)
    stripped_word = ''.join(re.findall(r'[\u05D0-\u05EA]', normalized_word))
    return unicodedata.normalize('NFC', stripped_word)

def process_word(token, verse_id, word_id, parent_element):
    parts = token.split('־')
    pe_count = 1  # Counter for 'פ' tags

    for part in parts:
        w = ET.SubElement(parent_element, 'w', id=f'verse{verse_id}_word{word_id}')

        alphabetic = strip_non_hebrew(part)
        non_alphabetic = ''.join(re.findall(r'[^\u05D0-\u05EA]', part))

        original = ET.SubElement(w, 'original')
        original.text = part
        stripped = ET.SubElement(w, 'stripped')
        stripped.text = alphabetic
        punctuation = ET.SubElement(w, 'punctuation')
        punctuation.text = non_alphabetic

        if "פ" in part:
            pe_tag = ET.SubElement(w, 'pe', id=f'verse{verse_id}_pe{pe_count}')
            pe_tag.text = "פ"
            pe_count += 1
        
        word_id += 1
    return word_id

def encode_tei_hebrew_word_details_enhanced(file_path, output_file):
    text = read_text_from_file(file_path)
    TEI = ET.Element('TEI', xmlns='http://www.tei-c.org/ns/1.0')
    text_element = ET.SubElement(TEI, 'text')
    body = ET.SubElement(text_element, 'body')

    chapter_id = 1
    verse_id = 1

    chapters = text.split('פרק')
    for chapter in chapters[1:]:
        div = ET.SubElement(body, 'div', type='chapter', id=f'chapter{chapter_id}')
        chapter_id += 1

        verses = re.split(r'(\[\פ\]|:)', chapter)
        for verse in verses:
            if verse.strip() and verse not in ['[פ]', ':']:
                p = ET.SubElement(div, 'p', type='verse', id=f'verse{verse_id}')
                word_id = 1

                tokens = verse.strip().split()
                for token in tokens:
                    word_id = process_word(token, verse_id, word_id, p)

                verse_id += 1

    tree = ET.ElementTree(TEI)
    with open(output_file, "w", encoding="utf-8") as f:
        tree.write(f, encoding="unicode")

# Specify the file paths
file_path = 'file.txt'  # Replace with your input file path
output_file = 'tei_hebrew_output_enhanced.xml'  # Replace with your output file path

# Run the function
encode_tei_hebrew_word_details_enhanced(file_path, output_file)
output_file

'tei_hebrew_output_enhanced.xml'

In [ ]:
[' ', '"', '$', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', 'E', 'I', 'T', '_', 'a', 'b', 'c', 'd', 'e', 'g', 'h', 'i', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', '֑', '֔', '֕', '֖', '֗', '֙', '֛', '֜', '֞', '֣', '֤', '֥', '֨', '֩', 'ְ', 'ֱ', 'ֲ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֻ', 'ּ', 'ֽ', '׀', 'ׁ', 'ׂ']

In [51]:
def extract_consecutive_non_hebrew_groups(file_path):
    text = read_text_from_file(file_path)
    non_hebrew_groups = set()

    # Using a regular expression to find sequences of non-Hebrew characters
    pattern = re.compile(r'([^\u05D0-\u05EA]{,2})')
    matches = pattern.findall(unicodedata.normalize('NFD', text))

    for match in matches:
        non_hebrew_groups.add(match.strip())

    return non_hebrew_groups

# Extract and print groups of consecutive non-Hebrew characters
file_path = 'file.txt'

consecutive_non_hebrew_groups = extract_consecutive_non_hebrew_groups(file_path)
print(sorted(consecutive_non_hebrew_groups))



['', '$', '$1', '$2', '$4', '2', ':', '[', ']', '֑', '֔', '֕', '֖', '֗', '֙', '֜', '֣', '֤', '֥', '֥$', '֨', '֩', 'ְ', 'ְ$', 'ְ֙', 'ְּ', 'ְׁ', 'ְׂ', 'ֱ', 'ֲ', 'ִ', 'ִ$', 'ִ֔', 'ִ֖', 'ִ֜', 'ִ֨', 'ִּ', 'ִֽ', 'ִׁ', 'ֵ', 'ֵ$', 'ֵ֔', 'ֵ֖', 'ֵ֗', 'ֵ֛', 'ֵ֣', 'ֵ֤', 'ֵ֨', 'ֵּ', 'ֵֽ', 'ֵׁ', 'ֶ', 'ֶ֑', 'ֶ֙', 'ֶ֣', 'ֶ֤', 'ֶ֥', 'ֶּ', 'ֶֽ', 'ֶׁ', 'ַ', 'ַ֗', 'ַ֙', 'ַּ', 'ַׁ', 'ָ', 'ָ֑', 'ָ֔', 'ָ֖', 'ָ֗', 'ָ֛', 'ָ֜', 'ָ֞', 'ָ֣', 'ָ֥', 'ָ֨', 'ָּ', 'ָֽ', 'ֹ', 'ֹ֖', 'ֹ֣', 'ֹ֤', 'ֹ֨', 'ֹּ', 'ֹׂ', 'ֻ', 'ּ', 'ּ֣', 'ֽ', '־', '־$', '׀', 'ׁ', 'ׂ֖', '\ufeff']
